In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [2]:
titanic_df = pd.read_csv('datasets/titanic_processed.csv')

titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,14.0,0,0,7.8542,0,0,1
1,1,1,1,28.0,0,0,26.5500,0,0,1
2,1,1,0,36.0,1,2,120.0000,0,0,1
3,0,3,1,17.0,1,0,7.0542,0,0,1
4,0,3,1,4.0,4,2,31.2750,0,0,1


In [3]:
FEATURES = list(titanic_df.columns[1:])

FEATURES

['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [4]:
result_dict = {}

In [5]:
def summarize_classification(y_test, y_pred):
    
    acc = accuracy_score(y_test, y_pred, normalize=True)
    num_acc = accuracy_score(y_test, y_pred, normalize=False)

    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return {'accuracy': acc, 
            'precision': prec,
            'recall':recall, 
            'accuracy_count':num_acc}

In [6]:
def build_model(classifier_fn,                
                name_of_y_col, 
                names_of_x_cols, 
                dataset, 
                test_frac=0.2):
    
    X = dataset[names_of_x_cols]
    Y = dataset[name_of_y_col]

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_frac)
       
    model = classifier_fn(x_train, y_train)
    
    y_pred = model.predict(x_test)

    y_pred_train = model.predict(x_train)
    
    train_summary = summarize_classification(y_train, y_pred_train)
    test_summary = summarize_classification(y_test, y_pred)
    
    pred_results = pd.DataFrame({'y_test': y_test,
                                 'y_pred': y_pred})
    
    model_crosstab = pd.crosstab(pred_results.y_pred, pred_results.y_test)
    
    return {'training': train_summary, 
            'test': test_summary,
            'confusion_matrix': model_crosstab}

In [7]:
def compare_results():
    for key in result_dict:
        print('Classification: ', key)

        print()
        print('Training data')
        for score in result_dict[key]['training']:
            print(score, result_dict[key]['training'][score])

        print()
        print('Test data')
        for score in result_dict[key]['test']:
            print(score, result_dict[key]['test'][score])
       
        print()

In [8]:
def logistic_fn(x_train, y_train):
    
    model = LogisticRegression(solver='liblinear')
    model.fit(x_train, y_train)
    
    return model

In [9]:
result_dict['survived ~ logistic'] = build_model(logistic_fn,
                                              'Survived',
                                               FEATURES,
                                               titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113



In [10]:
def linear_discriminant_fn(x_train, y_train, solver='svd'):
    
    model = LinearDiscriminantAnalysis(solver=solver)
    model.fit(x_train, y_train)
    
    return model

In [11]:
result_dict['survived ~ linear_discriminant_analysis'] = build_model(linear_discriminant_fn,
                                                                 'Survived',
                                                                  FEATURES,
                                                                  titanic_df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7961335676625659
precision 0.7752293577981652
recall 0.7161016949152542
accuracy_count 453

Test data
accuracy 0.7692307692307693
precision 0.7209302325581395
recall 0.5961538461538461
accuracy_count 110



/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [12]:
result_dict['survived ~ linear_discriminant_analysis'] = build_model(linear_discriminant_fn,
                                                                     'Survived',
                                                                      FEATURES[0:-1],
                                                                      titanic_df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7961335676625659
precision 0.7560975609756098
recall 0.7013574660633484
accuracy_count 453

Test data
accuracy 0.7832167832167832
precision 0.86
recall 0.6417910447761194
accuracy_count 112



In [13]:
def quadratic_discriminant_fn(x_train, y_train):
    
    model = QuadraticDiscriminantAnalysis()
    model.fit(x_train, y_train)
    
    return model

In [14]:
result_dict['survived ~ quadratic_discriminant_analysis'] = build_model(quadratic_discriminant_fn,
                                                                        'Survived',
                                                                        FEATURES[0:-1],
                                                                        titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7961335676625659
precision 0.7560975609756098
recall 0.7013574660633484
accuracy_count 453

Test data
accuracy 0.7832167832167832
precision 0.86
recall 0.6417910447761194
accuracy_count 112

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.7741935483870968
recall 0.7433628318584071
accuracy_count 462

Test data
accuracy 0.7832167832167832
precision 0.7719298245614035
recall 0.7096774193548387
accuracy_count 112



In [15]:
def sgd_fn(x_train, y_train, max_iter=1000, tol=1e-3):
    
    model = SGDClassifier(max_iter=max_iter, tol=tol)
    model.fit(x_train, y_train)
     
    return model

In [16]:
result_dict['survived ~ sgd'] = build_model(sgd_fn,
                                           'Survived',
                                            FEATURES,
                                            titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7961335676625659
precision 0.7560975609756098
recall 0.7013574660633484
accuracy_count 453

Test data
accuracy 0.7832167832167832
precision 0.86
recall 0.6417910447761194
accuracy_count 112

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.7741935483870968
recall 0.7433628318584071
accuracy_count 462

Test data
accuracy 0.7832167832167832
precision 0.7719298245614035
recall 0.7096774193548387
accuracy_count 112

Classification:  survived ~ sgd

Training data
accuracy 0.7363796133567663
precision 0.7547169811320755
recall 0.5194805194805194
accuracy_count 419

Test data
accuracy 

### LinearSVC

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

* SVC with a linear kernel
* dual=False when number of samples > number of features

In [17]:
def linear_svc_fn(x_train, y_train, C=1.0, max_iter=1000, tol=1e-3):
    
    model = LinearSVC(C=C, max_iter=max_iter, tol=tol, dual=False)
    model.fit(x_train, y_train) 
    
    return model

In [18]:
result_dict['survived ~ linear_svc'] = build_model(linear_svc_fn,
                                                  'Survived',
                                                   FEATURES,
                                                   titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7961335676625659
precision 0.7560975609756098
recall 0.7013574660633484
accuracy_count 453

Test data
accuracy 0.7832167832167832
precision 0.86
recall 0.6417910447761194
accuracy_count 112

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.7741935483870968
recall 0.7433628318584071
accuracy_count 462

Test data
accuracy 0.7832167832167832
precision 0.7719298245614035
recall 0.7096774193548387
accuracy_count 112

Classification:  survived ~ sgd

Training data
accuracy 0.7363796133567663
precision 0.7547169811320755
recall 0.5194805194805194
accuracy_count 419

Test data
accuracy 

In [19]:
def radius_neighbor_fn(x_train, y_train, radius=40.0):

    model = RadiusNeighborsClassifier(radius=radius)
    model.fit(x_train, y_train) 
    
    return model

In [20]:
result_dict['survived ~ radius_neighbors'] = build_model(radius_neighbor_fn,
                                                         'Survived',
                                                         FEATURES,
                                                         titanic_df)
compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7961335676625659
precision 0.7560975609756098
recall 0.7013574660633484
accuracy_count 453

Test data
accuracy 0.7832167832167832
precision 0.86
recall 0.6417910447761194
accuracy_count 112

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.7741935483870968
recall 0.7433628318584071
accuracy_count 462

Test data
accuracy 0.7832167832167832
precision 0.7719298245614035
recall 0.7096774193548387
accuracy_count 112

Classification:  survived ~ sgd

Training data
accuracy 0.7363796133567663
precision 0.7547169811320755
recall 0.5194805194805194
accuracy_count 419

Test data
accuracy 

max_depth = None [ If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples ]

max_features = None [None -- max_features=n_features, 
                     auto -- then max_features=sqrt(n_features), 
                     sqrt -- then max_features=sqrt(n_features), 
                     log2 -- then max_features=log2(n_features)]

In [21]:
def decision_tree_fn(x_train, y_train, max_depth=None, max_features=None): 
    
    model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
    model.fit(x_train, y_train)
    
    return model

In [22]:
result_dict['survived ~ decision_tree'] = build_model(decision_tree_fn,
                                                 'Survived',
                                                  FEATURES,
                                                  titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7961335676625659
precision 0.7560975609756098
recall 0.7013574660633484
accuracy_count 453

Test data
accuracy 0.7832167832167832
precision 0.86
recall 0.6417910447761194
accuracy_count 112

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.7741935483870968
recall 0.7433628318584071
accuracy_count 462

Test data
accuracy 0.7832167832167832
precision 0.7719298245614035
recall 0.7096774193548387
accuracy_count 112

Classification:  survived ~ sgd

Training data
accuracy 0.7363796133567663
precision 0.7547169811320755
recall 0.5194805194805194
accuracy_count 419

Test data
accuracy 

In [23]:
def naive_bayes_fn(x_train,y_train, priors=None):
    
    model = GaussianNB(priors=priors)
    model.fit(x_train, y_train)
    
    return model

In [24]:
result_dict['survived ~ naive_bayes'] = build_model(naive_bayes_fn,
                                                    'Survived',
                                                    FEATURES,
                                                    titanic_df)

compare_results()

Classification:  survived ~ logistic

Training data
accuracy 0.7908611599297012
precision 0.7729468599033816
recall 0.6896551724137931
accuracy_count 450

Test data
accuracy 0.7902097902097902
precision 0.7407407407407407
recall 0.7142857142857143
accuracy_count 113

Classification:  survived ~ linear_discriminant_analysis

Training data
accuracy 0.7961335676625659
precision 0.7560975609756098
recall 0.7013574660633484
accuracy_count 453

Test data
accuracy 0.7832167832167832
precision 0.86
recall 0.6417910447761194
accuracy_count 112

Classification:  survived ~ quadratic_discriminant_analysis

Training data
accuracy 0.81195079086116
precision 0.7741935483870968
recall 0.7433628318584071
accuracy_count 462

Test data
accuracy 0.7832167832167832
precision 0.7719298245614035
recall 0.7096774193548387
accuracy_count 112

Classification:  survived ~ sgd

Training data
accuracy 0.7363796133567663
precision 0.7547169811320755
recall 0.5194805194805194
accuracy_count 419

Test data
accuracy 